In [ ]:
import torch
import numpy as np
import cv2
import pylab
import matplotlib.pyplot as plt
import json
import pickle
%matplotlib inline
pylab.rcParams['figure.figsize'] = (24.0, 24.0)

In [ ]:
json_dir = 'demo_datasets/annotations/'
image_dir = 'demo_datasets/images/'
resPth = 'bbox_predict.pkl'
demo_cats = [{'id': 1, 'name': 'plane', 'supercategory': 'plane'}, 
             {'id': 2, 'name': 'baseball-diamond', 'supercategory': 'baseball-diamond'},
             {'id': 3, 'name': 'bridge', 'supercategory': 'bridge'}, 
             {'id': 4, 'name': 'ground-track-field', 'supercategory': 'ground-track-field'}, 
             {'id': 5, 'name': 'small-vehicle', 'supercategory': 'small-vehicle'}, 
             {'id': 6, 'name': 'large-vehicle', 'supercategory': 'large-vehicle'},
             {'id': 7, 'name': 'ship', 'supercategory': 'ship'}, 
             {'id': 8, 'name': 'tennis-court', 'supercategory': 'tennis-court'}, 
             {'id': 9, 'name': 'basketball-court', 'supercategory': 'basketball-court'},
             {'id': 10, 'name': 'storage-tank', 'supercategory': 'storage-tank'}, 
             {'id': 11, 'name': 'soccer-ball-field', 'supercategory': 'soccer-ball-field'}, 
             {'id': 12, 'name': 'roundabout', 'supercategory': 'roundabout'}, 
             {'id': 13, 'name': 'harbor', 'supercategory': 'harbor'}, 
             {'id': 14, 'name': 'swimming-pool', 'supercategory': 'swimming-pool'},
             {'id': 15, 'name': 'helicopter', 'supercategory': 'helicopter'}]

img_names = []
for file_name in os.listdir(image_dir):
    if 'png' in file_name:
        img_names.append(file_name)
demo_json = {
    'images': [],
    'annotations': [],
    'categories': demo_cats
}
for i, name in enumerate(img_names):
    img_info = {
        'file_name': name,
        'id': i + 1,
        'width': 1024,
        'height': 1024
    }
    demo_json['images'].append(img_info)
f = open(json_dir + 'demo.json', 'w')
json.dump(demo_json, f)
print(len(demo_json['images']))

In [ ]:
demo_json = json.load(open(json_dir + 'demo.json'))
results = pickle.load(open(resPth, 'rb'))

In [ ]:
vis_thr = 0.3
catid2label = {info['id']: info['name'] for info in demo_json['categories']}
random_pic_id = np.random.randint(0, len(results), 1)[0]
random_pic_id = 4
img_name = demo_json['images'][random_pic_id]['file_name']
img_path = image_dir + img_name
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
result = results[random_pic_id]
for label in range(len(result)):
    bboxes = result[label]
    for i in range(bboxes.shape[0]):
        convex_points = np.array(bboxes[i][:18]) 
        rbbox = np.array(bboxes[i][18:-1])
        score = str(round(bboxes[i][-1], 2))
        if float(score) < vis_thr:
            continue
        label_score = catid2label[label + 1] + ':' + score
        cv2.putText(img, label_score, (int(rbbox[0]), int(rbbox[1]) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1)
        for p in convex_points.reshape(-1, 2):
            cv2.circle(img, (int(p[0]), int(p[1])), 3, (255, 0, 0), -1)
        cv2.polylines(img, rbbox.reshape(1, -1, 2).astype(np.int32), True, (255, 0, 0))

plt.imshow(img)